In [1]:
import numpy as np
import os
import codecs
import pandas as pd

# 行と列を100まで表示する
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

os.chdir("../data")

In [2]:
with codecs.open('jader/drug202008.csv', "r", "Shift-JIS", "ignore") as file:
    drug = pd.read_table(file, delimiter=",")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (7,8,9,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
drug.columns

Index(['識別番号', '報告回数', '医薬品連番', '医薬品の関与', '医薬品（一般名）', '医薬品（販売名）', '経路',
       '投与開始日', '投与終了日', '投与量', '投与単位', '分割投与回数', '使用理由', '医薬品の処置',
       '再投与による再発の有無', 'リスク区分等'],
      dtype='object')

In [55]:
# 重複削除
drug.drop_duplicates(inplace=True)
sus_drug = drug[drug["医薬品の関与"] == "被疑薬"]
sus_drug = sus_drug[['識別番号', '医薬品連番', '医薬品（一般名）', '使用理由']]
sus_drug.drop_duplicates(inplace=True)

is_infliximab = (
    (sus_drug["医薬品（一般名）"] == "インフリキシマブ（遺伝子組換え）") |
    (sus_drug["医薬品（一般名）"] == "インフリキシマブ（遺伝子組換え）［後続１］") |
    (sus_drug["医薬品（一般名）"] == "インフリキシマブ（遺伝子組換え）［後続３］") |
    (sus_drug["医薬品（一般名）"] == "インフリキシマブ（遺伝子組換え）［後続２］") |
    (sus_drug["医薬品（一般名）"] == "INFLIXIMAB") |
    (sus_drug["医薬品（一般名）"] == "ＩＮＦＬＩＸＩＭＡＢ") |
    (sus_drug["医薬品（一般名）"] == "インフリキシマブ（遺伝子組換え）［インフリキシマブ後続３］") |
    (sus_drug["医薬品（一般名）"] == "INFLIXIMAB (NGX)")
)
is_etanercept = (
    (sus_drug["医薬品（一般名）"] == "エタネルセプト（遺伝子組換え）") |
    (sus_drug["医薬品（一般名）"] == "エタネルセプト（遺伝子組換え）［後続１］") |
    (sus_drug["医薬品（一般名）"] == "ETANERCEPT (NGX)") |
    (sus_drug["医薬品（一般名）"] == "エタネルセプト（遺伝子組換え）［後続２］") |
    (sus_drug["医薬品（一般名）"] == "エタネルセプト") |
    (sus_drug["医薬品（一般名）"] == "ETANERCEPT") |
    (sus_drug["医薬品（一般名）"] == "etanercept")
)
is_adalimumab = (
    (sus_drug["医薬品（一般名）"] == "ADALIMUMAB(GENETICAL RECOMBINATION)") |
    (sus_drug["医薬品（一般名）"] == "Adalimumab") |
    (sus_drug["医薬品（一般名）"] == "アダリムマブ（遺伝子組換え）") |
    (sus_drug["医薬品（一般名）"] == "アダリムマブ") |
    (sus_drug["医薬品（一般名）"] == "Humira")  # 追加
)
is_golimumab = (
    (sus_drug["医薬品（一般名）"] == "ゴリムマブ（遺伝子組換え）") |
    (sus_drug["医薬品（一般名）"] == "ゴリムマブ")
)
is_certolizumab = (
    (sus_drug["医薬品（一般名）"] == "セルトリズマブペゴル") |
    (sus_drug["医薬品（一般名）"] == "セルトリズマブ　ペゴル（遺伝子組換え）") |
    (sus_drug["医薬品（一般名）"] == "ＣＥＲＴＯＬＩＺＵＭＡＢ　ＰＥＧＯＬ")  # 追加
)

sus_drug['Infliximab'] = is_infliximab
sus_drug['Etanercept'] = is_etanercept
sus_drug['Adalimumab'] = is_adalimumab
sus_drug['Golimumab'] = is_golimumab
sus_drug['Certolizumab'] = is_certolizumab
sus_drug = sus_drug.replace(False, 0).replace(True, 1)

is_tnf = (
    (sus_drug['Infliximab'] == 1) |
    (sus_drug['Etanercept'] == 1) |
    (sus_drug['Golimumab'] == 1) |
    (sus_drug['Adalimumab'] == 1) |
    (sus_drug['Certolizumab'] == 1)
)

sus_drug["is_tnf"] = is_tnf
sus_drug.drop(['医薬品連番', '医薬品（一般名）', '使用理由'], axis=1, inplace=True)
sus_drug = sus_drug.drop_duplicates().replace(False, 0).replace(True, 1)
sus_drug = sus_drug[sus_drug["is_tnf"] == 1]

In [67]:
sus_drug.groupby("識別番号").max().to_csv('target/tnf_druger.csv', encoding='shift-jis')

Infliximab      14595
Etanercept      14595
Adalimumab      14595
Golimumab       14595
Certolizumab    14595
is_tnf          14595
dtype: int64

In [69]:
drug = pd.read_csv('target/tnf_druger.csv', encoding='shift-jis')

In [85]:
with codecs.open('jader/demo202008.csv', "r", "Shift-JIS", "ignore") as file:
    demo = pd.read_table(file, delimiter=",")
# 重複削除
demo.drop_duplicates(inplace=True)
demo1 = demo[['識別番号', '性別', '年齢', '報告年度・四半期']]
outer_drug_demo = pd.merge(demo1, drug, on="識別番号", how='outer')

# 欠損値の0埋め
outer_drug_demo[
    ['Infliximab', 'Etanercept', 'Adalimumab',
        'Golimumab', 'Certolizumab', 'is_tnf']
].fillna(0)
# concat 横結合：レコード数が同じことに注意
outer_drug_demo = pd.concat([outer_drug_demo[['識別番号', '性別', '年齢', '報告年度・四半期']], outer_drug_demo[[
                            'Infliximab', 'Etanercept', 'Adalimumab', 'Golimumab', 'Certolizumab', 'is_tnf']].fillna(0)], axis=1)

In [86]:
with codecs.open('jader/reac202008.csv', "r", "Shift-JIS", "ignore") as file:
    reac = pd.read_table(file, delimiter=",")

# 重複削除
reac1 = reac[['識別番号', '有害事象']]
reac1.drop_duplicates(inplace=True)
jader = pd.merge(reac1, outer_drug_demo, on='識別番号', how='outer')

<ipython-input-86-e6cc1551e2e1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reac1.drop_duplicates(inplace=True)


In [100]:
jader.to_csv('target/data.csv', encoding='shift-jis')

In [93]:
# 年度ごとの集計
jader[jader.is_tnf == 1].groupby('報告年度・四半期').sum().to_csv('target/count_years.csv', encoding='shift-jis')

In [99]:
# tnfαを飲んだ人の有害事象の集計
jader[jader.is_tnf == 1].groupby('有害事象').sum().sort_values('is_tnf', ascending=False)['is_tnf'].to_csv('target/count_side_effect.csv', encoding='shift-jis')

In [152]:
list(jader[jader.is_tnf == 1].groupby('有害事象').sum().sort_values('is_tnf', ascending=False)['is_tnf'].index[:10])

['肺炎',
 '間質性肺疾患',
 'ニューモシスチス・イロベチイ肺炎',
 '帯状疱疹',
 '発熱',
 '敗血症',
 '肺結核',
 '細菌性肺炎',
 'リンパ腫',
 '蜂巣炎']

In [138]:
jader[jader.is_tnf == 1].columns

Index(['識別番号', '有害事象', '性別', '年齢', '報告年度・四半期', 'Infliximab', 'Etanercept',
       'Adalimumab', 'Golimumab', 'Certolizumab', 'is_tnf'],
      dtype='object')

NameError: name 'df' is not defined

In [169]:
import math
import csv

side_effect = '肺炎'
drug = 'Infliximab'

# 書き込み用CSVファイルの作成
with open('target/ror.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['drug', 'side_effect', 'a', 'b', 'c', 'd', 'ror', '95%CI+', '95%CI-'])

def ror(df, side_effect, drug):
    total = df['識別番号'].nunique()
    try:
        a = df[(df['有害事象']==side_effect) & (df[drug]==1)]['識別番号'].nunique()
        b = df[df[drug]==1]['識別番号'].nunique() - a
        c = df[(df['有害事象']==side_effect) & (df[drug]!=1)]['識別番号'].nunique()
        d = total - (a+b+c)
        ROR = (a/b)/(c/d)
        ci_95m = math.exp(math.log(ROR)-1.96*(1/a+1/b+1/c+1/d)**0.5)
        ci_95p = math.exp(math.log(ROR)+1.96*(1/a+1/b+1/c+1/d)**0.5)
        
        # 追記モード
        with open('target/ror.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([drug, side_effect, a, b, c, d, ROR, ci_95m, ci_95p])
        print(a,b,c,d)
        print(ROR, ci_95m, ci_95p)
    except ValueError:
        print('バグった', side_effect, drug)
    
drug_list = ['Infliximab', 'Etanercept',　'Adalimumab', 'Golimumab', 'Certolizumab', 'is_tnf']

def get_se_list(drug):
    return list(jader[jader.is_tnf == 1].groupby('有害事象').sum().sort_values('is_tnf', ascending=False)[drug].index[:30])

# ループで回して全パターンを実行する
for d in drug_list:
    print('***********************', d, '***********************')
    for s in get_se_list(drug):
        ror(jader, s, d)


['肺炎', '間質性肺疾患', 'ニューモシスチス・イロベチイ肺炎', '帯状疱疹', '発熱', '敗血症', '肺結核', '細菌性肺炎', 'リンパ腫', '蜂巣炎', '注入に伴う反応', 'リンパ増殖性障害', '汎血球減少症', '器質化肺炎', '播種性結核', '結核', '乳癌', '肝機能異常', '発疹', '細菌性関節炎', '肺の悪性新生物', '白血球数減少', '非定型マイコバクテリア感染', '腎盂腎炎', '結腸癌', 'びまん性大細胞型Ｂ細胞性リンパ腫', '血小板数減少', '尿路感染', '関節リウマチ', '結核性胸膜炎']
*********************** Infliximab
425 4700 13123 626409
4.316342834814951 3.9025688388638335 4.773987657084459
397 4728 29035 610497
1.765528541170773 1.592586427516777 1.9572507813901743
400 4725 2907 636625
18.53944956800134 16.635325858489477 20.66152434934478
106 5019 2070 637462
6.5038815785041 5.3401164771297465 7.921264595700547
126 4999 13302 626230
1.1865999722271925 0.9934837161235536 1.417254728223917
122 5003 4751 634781
3.258128557931543 2.7163758042440063 3.9079282341654666
134 4991 527 639005
32.554488021512725 26.876552264014975 39.43194349975369
93 5032 1503 638029
7.845556502257806 6.35144429151192 9.691143306157773
113 5012 1284 638248
11.207063172995403 9.227818599854762 13.61082942

8 1394 425 642830
8.680293695670521 4.3043564796463585 17.50493923990418
12 1390 873 642382
6.3525130411135 3.584955386386929 11.25660366395466
21 1381 17481 625774
0.5443482840436545 0.35367574659590306 0.8378155901083879
9 1393 9439 633816
0.43383901481892234 0.22518403998476408 0.8358331735756577
15 1387 389 642866
17.8725143315732 10.64239262176896 30.01456344308362
18 1384 819 642436
10.201931016959918 6.3762671799020865 16.322935275181973
18 1384 12041 631214
0.6817897712680135 0.4281188986837285 1.0857668129924984
12 1390 584 642671
9.50040898787819 5.351251061559781 16.866667233259733
20 1382 873 642382
10.648809849430494 6.814520690596214 16.640517559189718
11 1391 1023 642232
4.964572559387151 2.734289343247532 9.01403531352191
3 1399 1155 642100
1.1921316710451806 0.383450060039137 3.7062920813310645
17 1385 17393 625862
0.44167542401077986 0.2737074016401785 0.7127216107643108
10 1392 1254 642001
3.677891666208363 1.9696354871112702 6.867710902286742
14 1388 468 642787
13.8

## ここから下は使用理由をカウントするときに使うかも、とりあえずは保留する

In [58]:
sus_drug.count()

識別番号            14906
Infliximab      14906
Etanercept      14906
Adalimumab      14906
Golimumab       14906
Certolizumab    14906
is_tnf          14906
dtype: int64

In [76]:
sus_drug['使用理由']

KeyError: '使用理由'

In [47]:
sus_drug.columns

Index(['識別番号', '使用理由', 'Infliximab', 'Etanercept', 'Adalimumab', 'Golimumab',
       'Certolizumab', 'is_tnf'],
      dtype='object')

In [9]:
def split_df_by_semicolon(df, var):
    return df[var].str.split(';', expand=True)

In [49]:
tmp = pd.concat(
    [split_df_by_semicolon(sus_drug, '使用理由'), 
     sus_drug[['識別番号', 'Infliximab', 'Etanercept','Adalimumab', 'Golimumab', 'Certolizumab', 'is_tnf']]], 
    axis=1
)

In [51]:
# 使用理由が重複している行については一番前のものが主であると判断し、それをカウントした！
tmp.drop(columns=[1,2,3,4,5,6,7,8], inplace=True)

In [52]:
tmp

,0,識別番号,Infliximab,Etanercept,Adalimumab,Golimumab,Certolizumab,is_tnf
34,クローン病,AB-04000010,1.0,0.0,0.0,0.0,0.0,1.0
38,関節リウマチ,AB-04000011,1.0,0.0,0.0,0.0,0.0,1.0
750,関節リウマチ,AB-04000123,1.0,0.0,0.0,0.0,0.0,1.0
2064,関節リウマチ,AB-04000352,1.0,0.0,0.0,0.0,0.0,1.0
2650,関節リウマチ,AB-04000452,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
3683266,治療変更(エンブレル皮下注用25mgを注射していたが高価なので変更しました),CA-19104618,0.0,1.0,0.0,0.0,0.0,1.0
3684361,関節リウマチ(関節リウマチ),CA-19105012,0.0,0.0,0.0,1.0,0.0,1.0
3684438,関節リウマチ(RA),CA-19105062,1.0,0.0,0.0,0.0,0.0,1.0
3685960,潰瘍性大腸炎(潰瘍性大腸炎),CA-19105997,0.0,0.0,0.0,1.0,0.0,1.0


In [53]:
tmp.rename(columns={0:'reason'}, inplace=True)

In [40]:
tmp.to_csv('target/check_tnf_drug_1012.csv', encoding='shift-jis', index=False)

In [42]:
tmp.reason.value_counts().to_csv('target/tnf_reason.csv')